In [1]:
import re
import string
from collections import Counter
import squarify
import matplotlib.pyplot as plt

from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import spacy
from spacy.tokenizer import Tokenizer

from bs4 import BeautifulSoup
import html as ihtml

import requests
import sqlite3

In [2]:
df = pd.read_csv('https://raw.githubusercontent.com/JimKing100/Hackathon/master/data/techsearch.csv')
df = df.drop(df.columns[0], axis=1)

In [3]:
def clean_text(text):
    text = text.replace('\n', ' ')                # remove newline
    text = BeautifulSoup(text, "lxml").get_text() # remove html
    text = text.replace('/', ' ')                 # remove forward slashes
    text = re.sub(r'[^a-zA-Z ^0-9]', '', text)    # letters and numbers only
    text = text.lower()                           # lower case
    text = re.sub(r'(x.[0-9])', '', text)         # remove special characters
    return text

df['description'] = df.apply(lambda x: clean_text(x['description']), axis=1)

In [4]:
nlp = spacy.load("en_core_web_lg")
tokenizer = Tokenizer(nlp.vocab)
STOP_WORDS = nlp.Defaults.stop_words.union(['year'])

In [5]:
# Tokenizer pipe removing stop words and blank words and lemmatizing
tokens = []

for doc in tokenizer.pipe(df['description'], batch_size=500):
    
    doc_tokens = []
    for token in doc:
        if (token.lemma_ not in STOP_WORDS) & (token.text != ' '):
            doc_tokens.append(token.lemma_)

    tokens.append(doc_tokens)

df['tokens'] = tokens

In [6]:
df.head()

,job_title,company,location,description,counts,city,job,low_salary,high_salary,tokens
0,Data Scientist,Pactera,None,looking fora data scientis architect who has 8...,1319,San Jose,data scientist,NaN,NaN,"[look, forum, datum, scientis, architect, 8, y..."
1,Data scientist - Global Sales,PayPal,None,responsibilities provide business requirement...,1319,San Jose,data scientist,NaN,NaN,"[responsibility, provide, business, requiremen..."
2,Data Scientist,Palo Verde Consulting,"Campbell, CA 95008 (Central Campbell area)",job title data scientistlocation campbell ca 9...,1319,San Jose,data scientist,150000.0,210000.0,"[job, title, datum, scientistlocation, campbel..."
3,Data Scientist,Spry Health,"Palo Alto, CA",spry healths mission is to build the worlds la...,1319,San Jose,data scientist,100000.0,135000.0,"[spry, healths, mission, build, world, large, ..."
4,Data Scientist (All Levels) - Santa Clara,LeanTaaS,"Santa Clara, CA 95050",help build technology that saves lives were a...,1319,San Jose,data scientist,NaN,NaN,"[help, build, technology, save, live, fast, gr..."


In [7]:
tech_terms = ['python', 'r', 'sql', 'hadoop', 'spark', 'java', 'sas', 'tableau',
              'hive', 'scala', 'aws', 'c', 'c++', 'matlab', 'tensorflow', 'excel',
              'nosql', 'linux', 'azure', 'scikit', 'machine learning', 'statistic',
              'analysis', 'computer science', 'visual', 'ai', 'deep learning',
              'nlp', 'natural language processing', 'neural network', 'mathematic',
              'database', 'oop', 'blockchain',
              'html', 'css', 'javascript', 'jquery', 'git', 'photoshop', 'illustrator',
              'word press', 'seo', 'responsive design', 'php', 'mobile', 'design', 'react',
              'security', 'ruby', 'fireworks', 'json', 'node', 'express', 'redux', 'ajax',
              'java', 'api', 'state management',
              'wireframe', 'ui prototype', 'ux writing', 'interactive design',
              'metric', 'analytic', 'ux research', 'empathy', 'collaborate', 'mockup', 
              'prototype', 'test', 'ideate', 'usability', 'high-fidelity design',
              'framework',
              'swift', 'xcode', 'spatial reasoning', 'human interface', 'core data',
              'grand central', 'network', 'objective-c', 'foundation', 'uikit', 
              'cocoatouch', 'spritekit', 'scenekit', 'opengl', 'metal', 'api', 'iot',
              'karma']

In [8]:
df['tokens_filtered'] = df.apply(lambda x: list(set(x['tokens']) & set(tech_terms)), axis=1)

In [9]:
df.head()

,job_title,company,location,description,counts,city,job,low_salary,high_salary,tokens,tokens_filtered
0,Data Scientist,Pactera,None,looking fora data scientis architect who has 8...,1319,San Jose,data scientist,NaN,NaN,"[look, forum, datum, scientis, architect, 8, y...","[design, network, r, python]"
1,Data scientist - Global Sales,PayPal,None,responsibilities provide business requirement...,1319,San Jose,data scientist,NaN,NaN,"[responsibility, provide, business, requiremen...","[foundation, collaborate, python, tableau, ana..."
2,Data Scientist,Palo Verde Consulting,"Campbell, CA 95008 (Central Campbell area)",job title data scientistlocation campbell ca 9...,1319,San Jose,data scientist,150000.0,210000.0,"[job, title, datum, scientistlocation, campbel...","[network, python]"
3,Data Scientist,Spry Health,"Palo Alto, CA",spry healths mission is to build the worlds la...,1319,San Jose,data scientist,100000.0,135000.0,"[spry, healths, mission, build, world, large, ...","[foundation, git, python, analytic, analysis, ..."
4,Data Scientist (All Levels) - Santa Clara,LeanTaaS,"Santa Clara, CA 95050",help build technology that saves lives were a...,1319,San Jose,data scientist,NaN,NaN,"[help, build, technology, save, live, fast, gr...","[python, analytic, test, analysis, design, sql]"


In [10]:
# Create a count function
def count(docs):

        word_counts = Counter()
        appears_in = Counter()
        
        total_docs = len(docs)

        for doc in docs:
            word_counts.update(doc)
            appears_in.update(set(doc))

        temp = zip(word_counts.keys(), word_counts.values())
        
        wc = pd.DataFrame(temp, columns = ['word', 'count'])

        wc['rank'] = wc['count'].rank(method='first', ascending=False)
        total = wc['count'].sum()

        wc['pct_total'] = wc['count'].apply(lambda x: x / total)
        
        wc = wc.sort_values(by='rank')
        wc['cul_pct_total'] = wc['pct_total'].cumsum()

        t2 = zip(appears_in.keys(), appears_in.values())
        ac = pd.DataFrame(t2, columns=['word', 'appears_in'])
        wc = ac.merge(wc, on='word')

        wc['appears_in_pct'] = wc['appears_in'].apply(lambda x: x / total_docs)
        
        return wc.sort_values(by='rank')

In [24]:
def populate_df(title, city):
    j_title = df['job'] == title
    j_city = df['city'] == city
    subset_df = df[j_title & j_city]
    subset_df = subset_df.reset_index()
    
    wc = count(subset_df['tokens_filtered'])
    skills = wc['word'][:10]
    
    if subset_df.shape[0] > 0:
        data = {'job': title,
                'city': city,
                'counts': subset_df['counts'][0],
                'low_salary': subset_df['low_salary'].mean(),
                'high_salary': subset_df['high_salary'].mean(),
                'skills': list(skills)}
    else:
        data = {'job': title,
                'city': city,
                'counts': 0,
                'low_salary': None,
                'high_salary': None,
                'skills': []}
    
    return data
    

In [25]:
final_df = pd.DataFrame(columns=['job', 'city', 'counts', 'low_salary', 'high_salary', 'skills'])

results = populate_df('data scientist', 'San Jose')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'San Francisco')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Seattle')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Washington')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'New York')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Baltimore')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Boulder')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'San Diego')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Denver')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Huntsville')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Colorado Springs')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Houston')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Trenton')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Dallas')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Columbus')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Austin')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Philadelphia')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Durham')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Raleigh')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('data scientist', 'Atlanta')
final_df = final_df.append(results, ignore_index=True)

In [26]:
results = populate_df('web developer', 'San Jose')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'San Francisco')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Seattle')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Washington')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'New York')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Baltimore')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Boulder')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'San Diego')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Denver')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Huntsville')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Colorado Springs')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Houston')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Trenton')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Dallas')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Columbus')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Austin')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Philadelphia')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Durham')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Raleigh')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('web developer', 'Atlanta')
final_df = final_df.append(results, ignore_index=True)

In [27]:
results = populate_df('ux designer', 'San Jose')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'San Francisco')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Seattle')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Washington')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'New York')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Baltimore')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Boulder')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'San Diego')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Denver')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Huntsville')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Colorado Springs')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Houston')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Trenton')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Dallas')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Columbus')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Austin')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Philadelphia')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Durham')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Raleigh')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ux designer', 'Atlanta')
final_df = final_df.append(results, ignore_index=True)

In [28]:
results = populate_df('ios developer', 'San Jose')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'San Francisco')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Seattle')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Washington')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'New York')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Baltimore')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Boulder')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'San Diego')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Denver')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Huntsville')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Colorado Springs')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Houston')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Trenton')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Dallas')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Columbus')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Austin')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Philadelphia')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Durham')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Raleigh')
final_df = final_df.append(results, ignore_index=True)
results = populate_df('ios developer', 'Atlanta')
final_df = final_df.append(results, ignore_index=True)

In [29]:
final_df

,job,city,counts,low_salary,high_salary,skills
0,data scientist,San Jose,1319,107500,152750,"[python, analysis, statistic, design, r, test,..."
1,data scientist,San Francisco,937,140000,225333,"[python, analysis, statistic, r, design, sql, ..."
2,data scientist,Seattle,1106,138760,186648,"[python, analysis, design, statistic, sql, r, ..."
3,data scientist,Washington,1181,77665.4,102974,"[python, analysis, statistic, analytic, r, des..."
4,data scientist,New York,818,50770.2,98770,"[python, design, statistic, sql, analysis, ana..."
5,data scientist,Baltimore,165,95292.7,125106,"[analysis, design, python, r, analytic, securi..."
6,data scientist,Boulder,161,5826,6117,"[python, analysis, statistic, design, analytic..."
7,data scientist,San Diego,164,73000,100000,"[python, analysis, design, analytic, statistic..."
8,data scientist,Denver,180,81451,105890,"[python, design, analysis, analytic, statistic..."
9,data scientist,Huntsville,22,120000,135000,"[analysis, analytic, design, security, metric,..."
